In [1]:
import os
import glob
import sys
import pandas as pd
import datetime as dt
import numpy as np
import math as ma
from matplotlib import pyplot as plt
from astropy import constants
from scipy import stats
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, precision_recall_curve
import itertools
from scipy.signal import savgol_filter, detrend
from scipy.stats.mstats import winsorize

In [2]:
tiles = 128
freqs = 640
freqs_hf = 768
part = 4

data_directory = "/Users/eormacstudio/Documents/20241116_observation_data_8s_before_with_cotter_time_corrections/uvfits/calibrated_fits/"
data_directory_after = "/Users/eormacstudio/Documents/20241112_observation_data_8s_after_with_cotter_time_corrections/uvfits/calibrated_fits/"
data_directory_higher_frequency_2s = "/Users/eormacstudio/Documents/20241123_contaminated_rfi_data_2s_with_no_flags/uvfits/calibrated_fits/"
data_directory_higher_frequency_8s = "/Users/eormacstudio/Documents/20241119_contaminated_rfi_data_8s_with_no_flags/uvfits/calibrated_fits/"
data_directory_model_sn1 = "/Users/eormacstudio/Documents/20240821_multiple_simulation_higher_thermal_noise_sn1/uvfits/calibrated_fits/"

In [11]:
# Import observation data

observation_file = "/Users/eormacstudio/Documents/winsorized_statistics/python_script/general_eor1_update_241010.csv"

df = pd.read_csv(observation_file, header=0, engine='python')
df = df[['obs_id', 'groupid', 'starttime_utc', 'local_sidereal_time_deg', 'duration',
        'int_time', 'freq_res', 'dataqualityname', 'bad_tiles', 'calibration',
        'calibration_delays', 'center_frequency_mhz', 'channel_center_frequencies_mhz_csv',
        'ra', 'ra_pointing', 'ra_phase_center', 'dec', 'dec_pointing', 'dec_phase_center',
        'deleted_flag', 'good_tiles', 'mode', 'sky_temp', 'stoptime_utc', 'total_tiles', 'gridpoint_name', 'gridpoint_number']]
df['date'] = df.starttime_utc.apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.000Z").date())
df['date_time'] = df.starttime_utc.apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.000Z"))
df['partition'] = pd.factorize(df['date'])[0] + 1
#df = df[(df['partition'] == 1) & (df['gridpoint_number'] == 0)].reset_index()

obs_list = df['obs_id'].to_list()

polr = ['XX', 'XY', 'YX', 'YY']

number_of_days = np.unique(df['partition'])
gn = df['gridpoint_number'].to_list()

In [12]:
# Group by (number_days, number_gridpoint)
grouped = df.groupby(["partition", "gridpoint_number"])["obs_id"].apply(list).reset_index()

# Prepare data for multiprocessing
task_list = [(row["partition"], row["gridpoint_number"], row["obs_id"]) for _, row in grouped.iterrows()]

In [ ]:
grouped

In [ ]:
[(obs_days, grid, obs_list, data_directory) for obs_days, grid, obs_list in task_list]

In [4]:
# Import observation data higher

observation_file = "/Users/eormacstudio/Documents/winsorized_statistics/python_script/rfi_contaminated_data_eor1_2016.csv"

df_high = pd.read_csv(observation_file, header=0, engine='python')
df_high = df_high[['obs_id', 'groupid', 'starttime_utc', 'local_sidereal_time_deg', 'duration',
        'int_time', 'freq_res', 'dataqualityname', 'bad_tiles', 'calibration',
        'calibration_delays', 'center_frequency_mhz', 'channel_center_frequencies_mhz_csv',
        'ra', 'ra_pointing', 'ra_phase_center', 'dec', 'dec_pointing', 'dec_phase_center',
        'deleted_flag', 'good_tiles', 'mode', 'sky_temp', 'stoptime_utc', 'total_tiles', 'gridpoint_name', 'gridpoint_number']]
df_high['date'] = df_high.starttime_utc.apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.000Z").date())
df_high['date_time'] = df_high.starttime_utc.apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.000Z"))
df_high['partition'] = pd.factorize(df_high['date'])[0] + 1

obs_list_high = df_high['obs_id'].to_list()

polr = ['XX', 'XY', 'YX', 'YY']

In [33]:
all_data = []

for i in range(len(obs_list_high)):
    f = fits.open(data_directory_higher_frequency_2s + "hyperdrive_solutions_%s.fits" %(obs_list_high[i]))

    data = f['SOLUTIONS'].data[:, [76], [76], ::2]
    #print(data.shape)
    #data = data[:, :, :, 0]
    data = data[:, :, 0]
    data = np.nan_to_num(data, nan=np.nanmedian(data))

    all_data.append(data)

combined_array = np.concatenate(all_data, axis=0)


In [ ]:
combined_array

In [ ]:
def flag_outliers(data, max_iters=100, tol=1e-6, epsilon=1e-8, k=3):
    """
    Flags outliers in a 3D data array (time, frequency, antennas) using
    exponential weighting for variance estimation.
    
    Parameters:
    - data: 3D numpy array of shape (time, frequency, antennas).
    - max_iters: int, maximum number of iterations for exponential weighting.
    - tol: float, tolerance for convergence in exponential weighting.
    - epsilon: float, minimum variance to avoid division by zero.
    - k: float, threshold multiplier for outlier detection (default: 3-sigma rule).

    Returns:
    - flag_array: 3D numpy array (same shape as `data`) with 1 for outliers, 0 otherwise.
    """
    time, antennas, freq = data.shape
    flag_array = np.zeros((time, antennas, freq), dtype=int)  # Initialize flag array
    outliers_data = []
    all_mu = []
    all_sigma_squared = []

    # Iterate over frequency and antennas
    for f in range(4): #freq):
        for a in range(2): #antennas):
            print("Process for: ", a, f)
            
            # Extract data for this frequency and antenna over time
            time_series = data[:, a, f]

            # Step 1: Apply exponential weighting algorithm
            # Initialization

            mu = np.median(time_series)
            sigma_squared = np.var(time_series) if np.var(time_series) > epsilon else epsilon
            
            for _ in range(max_iters):
                # Calculate weights
                q = (time_series - mu)**2 / sigma_squared
                weights = np.exp(-q / 4)
                
                # Update mean and variance
                new_mu = np.sum(time_series * weights) / np.sum(weights)
                new_sigma_squared = np.sum(((time_series - new_mu)**2 / sigma_squared - 2/3) * weights) / np.sum(weights)
                new_sigma_squared *= sigma_squared
                new_sigma_squared = max(new_sigma_squared, epsilon)

                # Convergence check
                if abs(new_mu - mu) < tol and abs(new_sigma_squared - sigma_squared) < tol:
                    break

                # Update for next iteration
                mu, sigma_squared = new_mu, new_sigma_squared
                all_mu.append(mu)
                all_sigma_squared.append(sigma_squared)

            # Step 2: Detect outliers
            sigma = np.sqrt(sigma_squared)
            outliers = np.abs(time_series - mu) > k * sigma
            flag_array[:, a, f] = outliers.astype(int)  # Update flag array

            outliers_data.append(outliers)

    return flag_array, outliers_data, all_mu, all_sigma_squared

# Flag outliers
flag_array, od, all_mu, all_sigma_squared= flag_outliers(combined_array)

# Find flagged locations
flagged_locations = np.argwhere(flag_array == 1)
print(f"Flagged Locations (time, frequency, antenna):\n{flagged_locations}")

In [ ]:
all_mu

In [ ]:
flag_array.shape

In [ ]:
flagged_locations.shape

In [ ]:
combined_array[flagged_locations[(flagged_locations[:,2] == f_idx) & (flagged_locations[:,1] == a_idx)][:,0], a_idx, f_idx]

In [ ]:
flagged_locations

In [ ]:
flagged_locations[flagged_locations[:, 1]]

In [ ]:
a_idx, f_idx = 4, 22  # Example frequency and antenna to visualize
plt.plot(combined_array[:, a_idx, f_idx], label=f"Freq={f_idx}, Antenna={a_idx}")
plt.scatter(
    flagged_locations[flagged_locations[:, 2] == f_idx][:, 0],
    combined_array[flagged_locations[flagged_locations[:, 2] == f_idx][:, 0], a_idx, f_idx],
    color="red", label="Outliers"
)
plt.title(f"Outlier Detection for Frequency={f_idx}, Antenna={a_idx}")
plt.xlabel("Time")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def exponential_weighting_outlier_detection(data, max_iters=100, tol=1e-6, epsilon=1e-8, k=5):
    """
    Implements the exponential weighting algorithm for robust variance estimation and outlier detection.

    Parameters:
    - data: array-like, the input data (e.g., real part of gain calibration solutions).
    - max_iters: int, maximum number of iterations for convergence.
    - tol: float, tolerance for convergence.
    - epsilon: float, minimum variance to avoid division by zero.
    - k: float, threshold for Z-score to classify outliers (default: 3-sigma rule).

    Returns:
    - mu_r: float, the robust mean.
    - sigma_r_squared: float, the robust variance.
    - z_scores: array-like, the Z-scores of the data points.
    - outliers: array-like, indices of the detected outliers.
    """
    # Step 1: Initialization
    mu_r = np.median(data)  # Robust initial guess for the mean
    sigma_r_squared = np.var(data) if np.var(data) > epsilon else epsilon  # Initial variance

    for iteration in range(max_iters):
        # Step 2: Compute weights based on current mean and variance
        q = (data - mu_r)**2 / sigma_r_squared
        weights = np.exp(-q / 4)

        # Step 3: Update mean
        new_mu_r = np.sum(data * weights) / np.sum(weights)

        # Step 4: Update variance
        new_sigma_r_squared = np.sum(((data - new_mu_r)**2 / sigma_r_squared - 2/3) * weights) / np.sum(weights)
        new_sigma_r_squared *= sigma_r_squared
        new_sigma_r_squared = max(new_sigma_r_squared, epsilon)  # Ensure variance is positive

        # Step 5: Check for convergence
        if abs(new_mu_r - mu_r) < tol and abs(new_sigma_r_squared - sigma_r_squared) < tol:
            break

        # Update for the next iteration
        mu_r, sigma_r_squared = new_mu_r, new_sigma_r_squared

    # Step 6: Compute Z-scores and identify outliers
    sigma_r = np.sqrt(sigma_r_squared)
    z_scores = (data - mu_r) / sigma_r
    outliers = np.where(np.abs(z_scores) > k)[0]  # Indices of outliers

    return mu_r, sigma_r_squared, z_scores, outliers

# Example Data: Simulated data with outliers

# Run the algorithm
mu_r, sigma_r_squared, z_scores, outliers = exponential_weighting_outlier_detection(combined_array)

# Print results
print("Robust Mean:", mu_r)
print("Robust Variance:", sigma_r_squared)
print("Outlier Indices:", len(outliers), outliers)
print("Outlier Values:", combined_array[outliers])
print("Z_score:", z_scores)

# Visualization
plt.figure(figsize=(10, 6))
plt.plot(combined_array, label="Data", marker="o", linestyle="-")
plt.scatter(outliers, combined_array[outliers], color="red", label="Outliers", zorder=5)
plt.axhline(mu_r, color="green", linestyle="--", label="Robust Mean")
plt.fill_between(
    range(len(combined_array)),
    mu_r - 3 * np.sqrt(sigma_r_squared),
    mu_r + 3 * np.sqrt(sigma_r_squared),
    color="green",
    alpha=0.2,
    label="3-Sigma Range"
)
plt.xlabel("Index")
plt.ylabel("Value")
plt.title("Exponential Weighting: Outlier Detection")
plt.legend()
plt.grid(True)
plt.show()


---

### Possibly correct

In [ ]:
all_data = []

for i in range(len(obs_list_high)):
    f = fits.open(data_directory_higher_frequency_2s + "hyperdrive_solutions_%s.fits" %(obs_list_high[i]))

    data = f['SOLUTIONS'].data[:, [20], [86], ::2]
    #print(data.shape)
    #data = data[:, :, :, 0]
    data = data[:, :, 0]
    data = np.nan_to_num(data, nan=np.nanmedian(data))

    all_data.append(data)

combined_array = np.concatenate(all_data, axis=0)


def exponential_weighting_outlier_detection(data, max_iters=100, tol=1e-6, epsilon=1e-8, k=4):
    """
    Implements the exponential weighting algorithm for robust variance estimation and outlier detection.

    Parameters:
    - data: array-like, the input data (e.g., real part of gain calibration solutions).
    - max_iters: int, maximum number of iterations for convergence.
    - tol: float, tolerance for convergence.
    - epsilon: float, minimum variance to avoid division by zero.
    - k: float, threshold for Z-score to classify outliers (default: 3-sigma rule).

    Returns:
    - mu_r: float, the robust mean.
    - sigma_r_squared: float, the robust variance.
    - z_scores: array-like, the Z-scores of the data points.
    - outliers: array-like, indices of the detected outliers.
    """
    # Step 1: Initialization
    mu_r = np.median(data)  # Robust initial guess for the mean
    sigma_r_squared = np.var(data) if np.var(data) > epsilon else epsilon  # Initial variance

    for iteration in range(max_iters):
        # Step 2: Compute weights based on current mean and variance
        q = (data - mu_r)**2 / sigma_r_squared
        weights = np.exp(-q / 4)

        # Step 3: Update mean
        new_mu_r = np.sum(data * weights) / np.sum(weights)

        # Step 4: Update variance
        new_sigma_r_squared = np.sum(((data - new_mu_r)**2 * weights) / np.sum(weights))
        new_sigma_r_squared = max(new_sigma_r_squared, epsilon)  # Ensure variance is positive

        # Step 5: Check for convergence
        if abs(new_mu_r - mu_r) < tol and abs(new_sigma_r_squared - sigma_r_squared) < tol:
            break

        # Update for the next iteration
        mu_r, sigma_r_squared = new_mu_r, new_sigma_r_squared

    # Step 6: Compute Z-scores and identify outliers
    sigma_r = np.sqrt(sigma_r_squared)
    z_scores = (data - mu_r) / sigma_r
    outliers = np.where(np.abs(z_scores) > k)[0]  # Indices of outliers

    return mu_r, sigma_r_squared, z_scores, outliers

# Example Data: Simulated data with outliers

# Run the algorithm
mu_r, sigma_r_squared, z_scores, outliers = exponential_weighting_outlier_detection(combined_array)

# Print results
print("Robust Mean:", mu_r)
print("Robust Variance:", sigma_r_squared)
print("Outlier Indices:", outliers)
print("Outlier Values:", combined_array[outliers])

# Visualization
plt.figure(figsize=(10, 6))
plt.plot(combined_array, label="Data", marker="o", linestyle="-")
plt.scatter(outliers, combined_array[outliers], color="red", label="Outliers", zorder=5)
plt.axhline(mu_r, color="green", linestyle="--", label="Robust Mean")
plt.fill_between(
    range(len(combined_array)),
    mu_r - 4 * np.sqrt(sigma_r_squared),
    mu_r + 4 * np.sqrt(sigma_r_squared),
    color="green",
    alpha=0.2,
    label="4-Sigma Range"
)
plt.xlabel("Index")
plt.ylabel("Value")
plt.title("Exponential Weighting: Outlier Detection")
plt.legend()
plt.grid(True)
plt.show()

---

### 3D Proccess

In [ ]:
data.shape

In [ ]:
sigma = 0
number_of_timeblocks = 56
all_data = []

for i in range(len(obs_list)):
    f = fits.open(data_directory_model_sn1 + "hyperdrive_solutions_%s_noise.fits" %(obs_list[i]))

    data = f['SOLUTIONS'].data[:, :, :, ::2]
    #data = data[:, :, :, :]

    all_data.append(data)

combined_array = np.concatenate(all_data, axis=0)


def exponential_weighting_outlier_detection_3d(data, obs_list, number_of_timeblocks, max_iters=100, tol=1e-6, epsilon=1e-8, k=sigma):
    """
    Implements the exponential weighting algorithm for robust variance estimation and outlier detection
    for 3D data (time, antenna, frequency).

    Parameters:
    - data: 3D numpy array, input data of shape (time, antenna, frequency).
    - max_iters: int, maximum number of iterations for convergence.
    - tol: float, tolerance for convergence.
    - epsilon: float, minimum variance to avoid division by zero.
    - k: float, threshold for Z-score to classify outliers (default: 3-sigma rule).

    Returns:
    - flag_array: 3D numpy array of the same shape as `data`, where 1 indicates an outlier and 0 otherwise.
    - stats: Dictionary containing robust mean and variance for each (antenna, frequency) combination.
    """
    time, antennas, frequencies, polarizations = data.shape
    flag_array = np.zeros_like(data, dtype=int)  # Initialize flag array
    stats = {}  # To store mean and variance for each (antenna, frequency)
    all_z_scores = {}
    print(time, antennas, frequencies, polarizations)

    # Iterate over antennas and frequencies
    for polarization in range(polarizations):
        for antenna in range(antennas):
            for frequency in range(frequencies):
                print(polarization, antenna, frequency)

                # Extract the time series for the current antenna and frequency
                time_series = data[:, antenna, frequency, polarization]
                time_series = np.nan_to_num(time_series, nan=np.nanmedian(time_series))

                # Step 1: Initialization
                mu_r = np.median(time_series)  # Robust initial guess for the mean
                sigma_r_squared = np.var(time_series) if np.var(time_series) > epsilon else epsilon  # Initial variance

                for iteration in range(max_iters):
                    # Step 2: Compute weights based on current mean and variance
                    q = (time_series - mu_r)**2 / sigma_r_squared
                    weights = np.exp(-q / 4)

                    # Step 3: Update mean
                    new_mu_r = np.sum(time_series * weights) / np.sum(weights)

                    # Step 4: Update variance
                    new_sigma_r_squared = np.sum(((time_series - new_mu_r)**2 * weights) / np.sum(weights))
                    new_sigma_r_squared = max(new_sigma_r_squared, epsilon)  # Ensure variance is positive

                    # Step 5: Check for convergence
                    if abs(new_mu_r - mu_r) < tol and abs(new_sigma_r_squared - sigma_r_squared) < tol:
                        break

                    # Update for the next iteration
                    mu_r, sigma_r_squared = new_mu_r, new_sigma_r_squared

                # Store robust statistics
                stats[(antenna, frequency, polarization)] = {'mean': mu_r, 'variance': sigma_r_squared}

                # Step 6: Compute Z-scores and identify outliers
                sigma_r = np.sqrt(sigma_r_squared)
                z_scores = (time_series - mu_r) / sigma_r
                outliers = np.abs(z_scores) > k
                
                all_z_scores[(antenna, frequency, polarization)] = {'data': time_series, 'z_score':z_scores, 'obs_id':np.repeat(obs_list, number_of_timeblocks, axis=0)}

                # Update flag array
                flag_array[:, antenna, frequency, polarization] = outliers.astype(int)

    return flag_array, stats, all_z_scores

# Run the algorithm
flag_array, stats, all_z_scores = exponential_weighting_outlier_detection_3d(combined_array, obs_list, number_of_timeblocks)

# Print results
print("Flagged Outliers (time, antenna, frequency):")
outlier_indices = np.argwhere(flag_array == 1)
#for idx in outlier_indices:
    #print(f"Time: {idx[0]}, Antenna: {idx[1]}, Frequency: {idx[2]}")

In [ ]:
# Visualization for a specific antenna and frequency
antenna, frequency, polarization = 75, 700, 0
plt.figure(figsize=(10, 6))
time_series = combined_array[:, antenna, frequency, polarization]
outliers = np.where(flag_array[:, antenna, frequency, polarization] == 1)[0]
plt.plot(time_series, label=f"Antenna {antenna}, Frequency {frequency}", marker="o")
plt.scatter(outliers, time_series[outliers], color="red", label="Outliers", zorder=5)
plt.axhline(stats[(antenna, frequency, polarization)]['mean'], color="green", linestyle="--", label="Robust Mean")
plt.fill_between(
    range(len(time_series)),
    stats[(antenna, frequency, polarization)]['mean'] - sigma * np.sqrt(stats[(antenna, frequency, polarization)]['variance']),
    stats[(antenna, frequency, polarization)]['mean'] + sigma * np.sqrt(stats[(antenna, frequency, polarization)]['variance']),
    color="green",
    alpha=0.2,
    label="%s-Sigma Range" %(sigma)
)
plt.xlabel("Time")
plt.ylabel("Value")
plt.title(f"Outlier Detection for Antenna {antenna}, Frequency {frequency}, Polarization {polarization}")
plt.legend()
plt.grid(True)
plt.show()

In [91]:
df_z_scores = pd.DataFrame(all_z_scores).T.reset_index()
df_z_scores = df_z_scores.rename(columns={'level_0': 'tile', 'level_1': 'frequency', 'level_2': 'polarization'})

In [ ]:
df_z_scores

In [92]:
df_outlier_statistics = pd.DataFrame(stats).T.reset_index()
df_outlier_statistics = df_outlier_statistics.rename(columns={'level_0': 'tile', 'level_1': 'frequency', 'level_2': 'polarization'})

In [ ]:
df_outlier_statistics

In [94]:
df_obs_id = pd.DataFrame(np.repeat(obs_list, 56, axis=0)).reset_index()
df_obs_id = df_obs_id.rename(columns={0: 'obs_id'})

df_outlier_indices = pd.DataFrame(outlier_indices)
df_outlier_indices = df_outlier_indices.rename(columns={0: 'index', 1:'tile', 2:'frequency', 3:'polarization'})
df_outlier_indices['obs_id'] = df_outlier_indices['index'].map(df_obs_id.set_index('index')['obs_id'].to_dict())

In [ ]:
df_outlier_indices

In [ ]:
df_data_higher_2s = pd.DataFrame()

for i in range(tiles):
    print('Tile: ', i)
    for j in range(7): #freqs_hf):

        #print('Tile: ', i, ' and Freqs: ', j)

        for k in range(len(obs_list_high)):
            f = fits.open(data_directory_higher_frequency_2s + "hyperdrive_solutions_%s.fits" %(obs_list_high[k]))

            data = f['SOLUTIONS'].data[:, i, j, ::2]

            a = pd.DataFrame(data[:, [0,1,2,3]], columns=polr)
            a['obs_id'] = obs_list_high[k]
            a['timeblocks'] = range(0, len(a))
            a['tile'] = i
            a['freq_channels'] = j
            df_data_higher_2s = pd.concat([df_data_higher_2s, a]).reset_index(drop = True)

df_data_higher_2s['start_time'] = df_data_higher_2s['obs_id'].map(df_high.set_index('obs_id').to_dict()['date_time'])
df_data_higher_2s['obs_time'] = df_data_higher_2s.apply(lambda x: x['start_time'] + pd.Timedelta(seconds=x['timeblocks'] * 2), axis=1)
df_data_higher_2s['partition'] = df_data_higher_2s['obs_id'].map(df_high.set_index('obs_id').to_dict()['partition'])
df_data_higher_2s['gridpoint_number'] = df_data_higher_2s['obs_id'].map(df_high.set_index('obs_id').to_dict()['gridpoint_number'])

In [ ]:
df_data_higher_2s

In [ ]:
56*128*7

In [ ]:
np.array(df_data_higher_2s['XX']).reshape(56, 128, 7)

In [12]:
df_data_tile = df_data_higher_2s[df_data_higher_2s['tile'] == 27].reset_index(drop=True)
df_data_tile = df_data_tile[~df_data_tile['XX'].isna()].reset_index(drop=True)
df_data_tile = df_data_tile.sort_values('obs_time').reset_index(drop=True)

In [ ]:
df_data_tile

In [ ]:
plt.scatter(df_data_tile['obs_time'], df_data_tile['XX'])

In [ ]:
detrended_data = detrend(df_data_tile['XX'])

winsorized_data = winsorize(detrended_data, limits=[0.05, 0.05])

plt.plot(winsorized_data, label="Preprocessed Data")

In [ ]:
plt.plot(detrended_data)

In [ ]:
import numpy as np

def flag_outliers(data, max_iters=100, tol=1e-6, epsilon=1e-8, k=3):
    """
    Flags outliers in a 3D data array (time, frequency, antennas) using
    exponential weighting for variance estimation.
    
    Parameters:
    - data: 3D numpy array of shape (time, frequency, antennas).
    - max_iters: int, maximum number of iterations for exponential weighting.
    - tol: float, tolerance for convergence in exponential weighting.
    - epsilon: float, minimum variance to avoid division by zero.
    - k: float, threshold multiplier for outlier detection (default: 3-sigma rule).

    Returns:
    - flag_array: 3D numpy array (same shape as `data`) with 1 for outliers, 0 otherwise.
    """
    time, freq, antennas = data.shape
    flag_array = np.zeros((time, freq, antennas), dtype=int)  # Initialize flag array

    # Iterate over frequency and antennas
    for f in range(freq):
        for a in range(antennas):
            # Extract data for this frequency and antenna over time
            time_series = data[:, f, a]

            # Step 1: Apply exponential weighting algorithm
            # Initialization
            mu = np.median(time_series)
            sigma_squared = np.var(time_series) if np.var(time_series) > epsilon else epsilon
            
            for _ in range(max_iters):
                # Calculate weights
                q = (time_series - mu)**2 / sigma_squared
                weights = np.exp(-q / 4)
                
                # Update mean and variance
                new_mu = np.sum(time_series * weights) / np.sum(weights)
                new_sigma_squared = np.sum(((time_series - new_mu)**2 / sigma_squared - 2/3) * weights) / np.sum(weights)
                new_sigma_squared *= sigma_squared
                new_sigma_squared = max(new_sigma_squared, epsilon)

                # Convergence check
                if abs(new_mu - mu) < tol and abs(new_sigma_squared - sigma_squared) < tol:
                    break

                # Update for next iteration
                mu, sigma_squared = new_mu, new_sigma_squared

            # Step 2: Detect outliers
            sigma = np.sqrt(sigma_squared)
            outliers = np.abs(time_series - mu) > k * sigma
            flag_array[:, f, a] = outliers.astype(int)  # Update flag array

    return flag_array

# Example 3D data: Simulated with random noise and outliers
np.random.seed(42)
time, freq, antennas = 100, 50, 5  # 100 time steps, 50 frequency channels, 5 antennas
data = np.random.normal(1, 0.05, (time, freq, antennas))

# Inject outliers
data[20, 10, 2] += 3  # Outlier in (time=20, freq=10, antenna=2)
data[50, 25, 1] -= 3  # Outlier in (time=50, freq=25, antenna=1)

# Flag outliers
flag_array = flag_outliers(data)

# Find flagged locations
flagged_locations = np.argwhere(flag_array == 1)
print(f"Flagged Locations (time, frequency, antenna):\n{flagged_locations}")

# Visualize flags for a specific frequency channel and antenna
import matplotlib.pyplot as plt

f_idx, a_idx = 10, 2  # Example frequency and antenna to visualize
plt.plot(data[:, f_idx, a_idx], label=f"Freq={f_idx}, Antenna={a_idx}")
plt.scatter(
    flagged_locations[flagged_locations[:, 1] == f_idx][:, 0],
    data[flagged_locations[flagged_locations[:, 1] == f_idx][:, 0], f_idx, a_idx],
    color="red", label="Outliers"
)
plt.title(f"Outlier Detection for Frequency={f_idx}, Antenna={a_idx}")
plt.xlabel("Time")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def process_observation_group(group, directory, number_of_timeblocks):
    """
    Function to process a group of observations in parallel.
    """
    number_days, number_gridpoint, observation_ids = group
    print(f"Processing: Days={number_days}, Gridpoints={number_gridpoint}, Observations={len(observation_ids)}")
    
    # Simulate processing (Replace with actual logic)
    for obs_id in observation_ids:
        obs_path = os.path.join(directory, f"{obs_id}.txt")
        print(f"Processing {obs_path} with {number_of_timeblocks} timeblocks")
        
    return f"Completed: Days={number_days}, Gridpoints={number_gridpoint}"

def main():
    """
    Main function to handle user input, load data, and process observations in parallel.
    """
    parser = argparse.ArgumentParser(description="Process observation data in parallel.")
    parser.add_argument("-d", "--directory", required=True, help="Directory containing observation data")
    parser.add_argument("-nt", "--number_of_timeblocks", type=int, required=True, help="Number of timeblocks")
    args = parser.parse_args()
    
    # Load observation ID dataframe (Simulated example)
    data = {
        "observation_id": [f"obs_{i}" for i in range(116)],
        "number_days": [1, 2, 3, 4] * 29,
        "number_gridpoint": [10, 20, 30] * 39
    }
    df = pd.DataFrame(data)
    
    # Group by (number_days, number_gridpoint)
    grouped = df.groupby(["number_days", "number_gridpoint"])["observation_id"].apply(list).reset_index()
    
    # Prepare data for multiprocessing
    task_list = [(row["number_days"], row["number_gridpoint"], row["observation_id"]) for _, row in grouped.iterrows()]
    
    # Run multiprocessing
    with mp.Pool(processes=mp.cpu_count()) as pool:
        results = pool.starmap(process_observation_group, [(group, args.directory, args.number_of_timeblocks) for group in task_list])
    
    # Print results
    for res in results:
        print(res)